In [1]:
import pandas as pd
import itertools
from scipy import stats

In [3]:
def get_gt(filename):
    with open(filename, "r") as fileHandler:
        lines = fileHandler.readlines()
    return lines

def get_pop(filename):
    phenotypes = []
    ids = []
    with open(filename, "r") as fileHandler:
        lines = fileHandler.readlines()
    for i in range(len(lines)):
        ids.append(lines[i].split()[0])
        phenotypes.append(lines[i].split()[2])
    return ids, phenotypes

def get_snp_ids(filename):
    snp_ids = ['IND', 'X', 'X', 'X', 'X', 'Pheno']
    with open(filename, "r") as fileHandler:
        lines = fileHandler.readlines()
    for i in range(len(lines)):
        snp_ids.append(lines[i].split()[1])
        snp_ids.append(lines[i].split()[1])
    return snp_ids

def get_ids(genotypes):
    ids = []
    for i in range(0, len(genotypes)):
        ids.append(genotypes[i].split()[0])
    return ids

def get_data_frame(genotypes, snp_ids):
    inds = []
    for i in range(0, len(genotypes)):
        inds.append(genotypes[i].split())
    df = pd.DataFrame(inds)
    df.columns = snp_ids
    return df

def which_minor_al(frq1, frq2, a1, a2):
    if frq1 < frq2:
        return frq1, a1
    else:
        return frq2, a2

def get_al_freq(snp_id, pop):
    aa = 0
    bb = 0
    ab = 0
    total = 2*len(pop[[snp_id]])
    al_1 = "1"
    al_2 = "2"

    for i in range(len(pop[[snp_id]])):
        
        if pop[[snp_id]].iloc[i].iloc[0] == '2' and pop[[snp_id]].iloc[i].iloc[1] == '2':
            aa += 1
        elif pop[[snp_id]].iloc[i].iloc[0] == '1' and pop[[snp_id]].iloc[i].iloc[1] == '1':
            bb += 1
        else:
            ab += 1

    obs_a = 2*aa + ab
    obs_b = 2*bb + ab

    f1 = (2*aa + ab) / total
    f2 = (2*bb + ab) / total

    return which_minor_al(f1, f2, al_1, al_2)

def get_missing(snp_id, df):
    missing_nu = 0
    for i in range(len(df[[snp_id]])):
        #if df[[snp_id]].iloc[i].all() == '0':
         #   missing_nu += 2
        if df[[snp_id]].iloc[i].any() == '0':
            missing_nu += 1
    return missing_nu, missing_nu / len(df[[snp_id]])

def missing_snps(person, nu_snps):
    missing = 0
    for i in range(nu_snps-6):
        print(i+6)
        if person[i+6] == '0' or person[i+7] == '0':
            missing += 1
        i += 2
    return missing

In [7]:
def chi_square_test(pop):
    aa_aff = 0
    bb_aff = 0
    ab_aff = 0

    aa_unaf = 0
    bb_unaf = 0
    ab_unaf = 0

    AFFECTED = '2'
    CONTROL = '1'

    for i in range(len(pop)):
        if pop.iloc[i].iloc[1] == '2' and pop.iloc[i].iloc[2] == '2' and pop.iloc[i].iloc[0] == AFFECTED:
            aa_aff += 1
        elif pop.iloc[i].iloc[1] == '1' and pop.iloc[i].iloc[2] == '1' and pop.iloc[i].iloc[0] == AFFECTED:
            bb_aff += 1
        elif pop.iloc[i].iloc[0] == AFFECTED:
            ab_aff += 1
        
        if pop.iloc[i].iloc[1] == '2' and pop.iloc[i].iloc[2] == '2' and pop.iloc[i].iloc[0] == CONTROL:
            aa_unaf += 1
        elif pop.iloc[i].iloc[1] == '1' and pop.iloc[i].iloc[2] == '1' and pop.iloc[i].iloc[0] == CONTROL:
            bb_unaf += 1
        elif pop.iloc[i].iloc[0] == CONTROL:
            ab_unaf += 1

    a = 2*aa_aff + ab_aff
    b = 2*bb_aff + ab_aff
    c = 2*aa_unaf + ab_unaf
    d = 2*bb_unaf + ab_unaf

    total = a + b + c + d

    #print (a, b, c, d)
    a_aff = a / total
    b_aff = b / total
    a_unaf = c / total
    b_unaf = d / total
    #print(a_aff, b_aff, a_unaf, b_unaf)

    p_aff = a_aff + b_aff
    p_unaf = a_unaf + b_unaf
    p_a = a_aff + a_unaf
    p_b = b_aff + b_unaf
    #print(p_aff, p_unaf, p_a, p_b)

    p_a_aff = p_aff * p_a
    p_b_aff = p_aff * p_b
    p_a_unaf = p_unaf * p_a
    p_b_unaf = p_unaf * p_b
    #print(p_a_aff, p_b_aff, p_a_unaf, p_b_unaf)

    exp_a_aff = total * p_a_aff
    exp_b_aff = total * p_b_aff
    exp_a_unaf = total * p_a_unaf
    exp_b_unaf = total * p_b_unaf
    #print(exp_a_aff, exp_b_aff, exp_a_unaf, exp_b_unaf)

    chi_a_aff = ((a - exp_a_aff) ** 2) / exp_a_aff
    chi_b_aff = ((b - exp_b_aff) ** 2) / exp_b_aff
    chi_a_unaf = ((c - exp_a_unaf) ** 2) / exp_a_unaf
    chi_b_unaf = ((d - exp_b_unaf) ** 2) / exp_b_unaf

    x_2 = chi_a_aff + chi_b_aff + chi_a_unaf + chi_b_unaf

    p_val = 1- stats.chi2.cdf(x_2, 1)
    print('Chi-square test (1 df): ', x_2)  
    print('P-value: ', p_val)

### Read in the categorical phenotypes

In [2]:
pop_file = pd.read_csv("pop.phe", sep='\s+', header=None)
pop_file.head()


,0,1,2
0,HCB181,1,1
1,HCB182,1,1
2,HCB183,1,1
3,HCB184,1,1
4,HCB185,1,1


In [4]:
# getting 
# IDs are the first column
# The phenotype encodings are the second column.
ids = pop_file[0]
pop =  pop_file[2]

### Duplicate SNP IDs
Since each SNP has to values (one for each chromosome copy) we need to provide each SNP ID with a extra character so we know what SNP we are looking at. Here we add _1 and _2 to each SNP ID.

In [7]:
snp_ids = pd.read_csv("hapmap1.map", sep='\s+', header=None)[1]
snp_ids_diploid = [(x+"_1", x+"_2") for x in list(snp_ids)]
snp_ids_diploid = list(itertools.chain(*snp_ids_diploid))
snp_ids_diploid[0:10]

['rs6681049_1',
 'rs6681049_2',
 'rs4074137_1',
 'rs4074137_2',
 'rs7540009_1',
 'rs7540009_2',
 'rs1891905_1',
 'rs1891905_2',
 'rs9729550_1',
 'rs9729550_2']

In [5]:
genotype = pd.read_csv("hapmap1.ped", sep="\s+", header=None)
genotype['populations'] = pop
genotype.iloc[:10, :10]

,0,1,2,3,4,5,6,7,8,9
0,HCB181,1,0,0,1,1,2,2,2,2
1,HCB182,1,0,0,1,1,2,2,1,2
2,HCB183,1,0,0,1,2,2,2,1,2
3,HCB184,1,0,0,1,1,2,2,1,2
4,HCB185,1,0,0,1,1,2,2,1,2
5,HCB186,1,0,0,1,1,2,2,2,2
6,HCB187,1,0,0,1,1,2,2,2,2
7,HCB188,1,0,0,1,1,2,2,1,2
8,HCB189,1,0,0,1,1,2,2,2,2
9,HCB190,1,0,0,1,1,2,2,2,2


We now need to adjust the column names to include the individual IDS, phenotypes, and the SNP IDs we just made.

In [10]:
col_names = ["individual", "X1", "X2", "X3", "X4", "phenotype"] + snp_ids_diploid + ['populations']

In [11]:
genotype.columns = col_names
genotype.iloc[:10, :10]

,individual,X1,X2,X3,X4,phenotype,rs6681049_1,rs6681049_2,rs4074137_1,rs4074137_2
0,HCB181,1,0,0,1,1,2,2,2,2
1,HCB182,1,0,0,1,1,2,2,1,2
2,HCB183,1,0,0,1,2,2,2,1,2
3,HCB184,1,0,0,1,1,2,2,1,2
4,HCB185,1,0,0,1,1,2,2,1,2
5,HCB186,1,0,0,1,1,2,2,2,2
6,HCB187,1,0,0,1,1,2,2,2,2
7,HCB188,1,0,0,1,1,2,2,1,2
8,HCB189,1,0,0,1,1,2,2,2,2
9,HCB190,1,0,0,1,1,2,2,2,2


### Rearrange the columns and set the index of the dataframe to be the individual IDs.

In [14]:
#### Here re-sort the column headers so that:
# 1- Individual is your index
# 2- pop and phenotype are first columns, followed by snp_ids
# 3- remove the x columns if there are not needed or rename them to their proper names if they are needed.
re_col_names = ['individual', 'populations', 'phenotype', 'X1', 'X2', 'X3', 'X4'] + snp_ids_diploid
genotype = genotype[re_col_names]
genotype = genotype.set_index('individual')
genotype.head()

,populations,phenotype,X1,X2,X3,X4,rs6681049_1,rs6681049_2,rs4074137_1,rs4074137_2,...,rs2269380_1,rs2269380_2,rs6151412_1,rs6151412_2,rs11912064_1,rs11912064_2,rs1001469_1,rs1001469_2,rs756638_1,rs756638_2
individual,,,,,,,,,,,,,,,,,,,,,
HCB181,1,1,1,0,0,1,2,2,2,2,...,2,2,2,2,2,2,1,2,2,2
HCB182,1,1,1,0,0,1,2,2,1,2,...,2,2,2,2,2,2,1,2,2,2
HCB183,1,2,1,0,0,1,2,2,1,2,...,1,1,2,2,2,2,2,2,1,2
HCB184,1,1,1,0,0,1,2,2,1,2,...,1,2,2,2,2,2,1,1,1,2
HCB185,1,1,1,0,0,1,2,2,1,2,...,1,2,2,2,2,2,1,2,2,2


The above restructring allows us to access individual data by using the individual's ID along with the attribute we want. For example, if we want to access individual HCB181 and look at there phenotype, population, and allele for SNP rs6681049 we could do the following:

In [31]:
genotype.loc['HCB181'][['populations', 'phenotype', 'rs6681049_1', 'rs6681049_2']]

populations    1
phenotype      1
rs6681049_1    2
rs6681049_2    2
Name: HCB181, dtype: int64

Here we can see that individual HCB181 belongs to population 1, has phenotype code 1, and the SNP has a homozygous genotype for the second variant.

In [44]:
from collections import Counter

We can now filter by each population and obtain SNP frequencies. 

In [70]:
# Filter by population 1 and 2
genotypes_pop_1 = genotype[genotype['populations'] == 1]
genotypes_pop_2 = genotype[genotype['populations'] == 2]

In [71]:
# Define SNP ID we want to look at
snp_id = 'rs6681049'

# Grab SNP values within each population
snp_genotype_pop_1 = genotypes_pop_1[snp_id+"_1"].astype('str') +  + genotypes_pop_1[snp_id+"_2"].astype('str')
snp_genotype_pop_2 = genotypes_pop_2[snp_id+"_1"].astype('str') +  + genotypes_pop_2[snp_id+"_2"].astype('str')

In [72]:
# Count the occurrences of each allele and obtain frequencies
genotype_counts_pop_1 = Counter(list(snp_genotype_pop_1))
genotype_counts_pop_2 = Counter(list(snp_genotype_pop_2))

count_aa_pop_1 = genotype_counts_pop_1['22']
count_bb_pop_1 = genotype_counts_pop_1['11']
count_ab_pop_1 = genotype_counts_pop_1['12']

count_aa_pop_2 = genotype_counts_pop_2['22']
count_bb_pop_2 = genotype_counts_pop_2['11']
count_ab_pop_2 = genotype_counts_pop_2['12']

In [76]:
# We then define the number of occurrences by twice the total since we are counting by pairs of alleles
total = count_aa_pop_1 + count_bb_pop_1 + count_ab_pop_1
A1_frequency = (2*count_aa_pop_1 + count_ab_pop_1) / (2*total)
A2_frequency = (2*count_bb_pop_1 + count_ab_pop_1) / (2*total)
print('Population 1 Allele 1 freq: ', A1_frequency)
print('Population 1 Allele 2 freq: ', A2_frequency)

Allele 1 freq:  0.7666666666666667
Allele 2 freq:  0.23333333333333334


In [77]:
A1_frequency = (2*count_aa_pop_2 + count_ab_pop_2) / (2*total)
A2_frequency = (2*count_bb_pop_2 + count_ab_pop_2) / (2*total)
print('Population 2 Allele 1 freq: ', A1_frequency)
print('Population 2 Allele 2 freq: ', A2_frequency)

Population 2 Allele 1 freq:  0.7888888888888889
Population 2 Allele 2 freq:  0.18888888888888888


When considering the frequency of SNPs we only consider the minor allele frequency which is the frequency of the allele that occurs less. Thus, in population 1 we see a minor allele frequency of 0.233 and for population 2 we see a minor allele frequency of 0.188 for this particular SNP.

In [92]:
# Perform Basic Association Test

# Filter by different phenotypes for specific SNP
GT_uf = genotype[genotype['phenotype'] == 1][[snp_id+'_1', snp_id+'_2']]
GT_af = genotype[genotype['phenotype'] == 2][[snp_id+'_1', snp_id+'_2']]

snp_GT_uf = GT_uf[snp_id+"_1"].astype('str') +  + GT_uf[snp_id+"_2"].astype('str')
snp_GT_af = GT_af[snp_id+"_1"].astype('str') +  + GT_af[snp_id+"_2"].astype('str')

# get MAF for affected and unaffected
GT_counts_uf = Counter(list(snp_GT_uf))
GT_counts_af = Counter(list(snp_GT_af))

count_aa_GT_uf = GT_counts_uf['22']
count_bb_GT_uf = GT_counts_uf['11']
count_ab_GT_uf = GT_counts_uf['12']
uf_total = 2*(count_aa_GT_uf + count_bb_GT_uf + count_ab_GT_uf)

count_aa_GT_af = GT_counts_af['22']
count_bb_GT_af = GT_counts_af['11']
count_ab_GT_af = GT_counts_af['12']
af_total = 2*(count_aa_GT_af + count_bb_GT_af + count_ab_GT_af)

a_uf = (2*count_aa_GT_uf + count_ab_GT_uf) / uf_total
b_uf = (2*count_bb_GT_uf + count_ab_GT_uf) / uf_total

a_af = (2*count_aa_GT_af + count_ab_GT_af) / af_total
b_af = (2*count_bb_GT_af + count_ab_GT_af) / af_total

print('MAF Unaffected: ', b_uf)
print('MAF Affected: ', b_af)

# perform test
p_aff = a_af + b_af
p_unaff = a_uf + b_uf
p_a = a_af + a_uf
p_b = b_af + b_uf

exp_a_af = p_a * p_aff
exp_a_uf = p_a * p_unaff
exp_b_af = p_b * p_aff
exp_b_uf = p_b * p_unaff

a_count_uf = a_uf * uf_total
a_count_af = a_af * af_total
b_count_uf = b_uf * uf_total
b_count_af = b_af * af_total



stats.chisquare([a_uf, b_uf, a_af, b_af], [exp_a_uf, exp_b_uf, exp_a_af, exp_b_af])

# get p-value



MAF Unaffected:  0.26666666666666666
MAF Affected:  0.1590909090909091


Power_divergenceResult(statistic=1.0172661229830218, pvalue=0.7970741635167385)

In [ ]:

maf_pop2, allele_pop2 = get_al_freq('rs6681049', df_pop2)
phe_unaf_frq, allele_phe1 = get_al_freq('rs6681049', df_pheno1)
phe_af_frq, allele_phe2 = get_al_freq('rs6681049', df_pheno2)

# Missing statistics
missing_nu, freq = get_missing('rs6704013', df)
print('rs6704013: # of missing: ', missing_nu, ' frequency: ', freq)

missing_nu, freq = get_missing('rs307347', df)
print('rs307347: # of missing: ', missing_nu, ' frequency: ', freq)

missing_nu, freq = get_missing('rs9439440', df)
print('rs9439440: # of missing: ', missing_nu, ' frequency: ', freq)

# Minor allele statistics
print()
print('SNP ID: rs6681049')
print('MAF Population 1: ', maf_pop1, "Minor allele: ", allele_pop1)
print('MAF Population 2: ', maf_pop2, "Minor allele: ", allele_pop2)
print('Frequency in unaffected: ', phe_unaf_frq, "Allele: ", allele_phe1)
print('Frequency in affected: ', phe_af_frq, "Allele: ", allele_phe2)

# Basic Association Test
print()
df_snp1 = df[['Pheno', 'rs6681049']]
df_snp2 = df[['Pheno', 'rs9585021']]
df_snp3 = df[['Pheno', 'rs2222162']]

print('Basic Association Test rs6681049: ')
chi_square_test(df_snp1)

print()
print('Basic Association Test rs9585021: ' )
chi_square_test(df_snp2)

print()
print('Basic Association Test rs2222162: ')
chi_square_test(df_snp3)

Note: The test for rs6958021 does not match the result in PLINK but the other SNPs do.